In [222]:
import pandas as pd
import numpy as np
from helpers import *
import os
import calendar
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from IPython.display import clear_output

months = {month: index for index, month in enumerate(calendar.month_name) if month}

In [223]:
all_teams = pd.read_csv("all_teams.csv")
all_teams = all_teams[["id", "full_name"]]
elo_ts = pd.read_csv("save_elo_ts.csv")
df = (
    elo_ts.merge(all_teams, left_on="TEAM_ID", right_on="id")
    .drop("id", axis=1)
    .rename(columns={"full_name": "TEAM"})
)
df = df.sort_values(by="DATE").reset_index(drop=True)
df.DATE = pd.to_datetime(df.DATE)
df["MONTH"] = df.DATE.dt.month_name()
df.head()

,DATE,SEASON,TEAM_ID,ELO,TEAM,MONTH
0,2003-10-05,2003-04,1610612762,1416.0,Utah Jazz,October
1,2003-10-05,2003-04,1610612742,1384.0,Dallas Mavericks,October
2,2003-10-06,2003-04,1610612763,1416.0,Memphis Grizzlies,October
3,2003-10-06,2003-04,1610612749,1384.0,Milwaukee Bucks,October
4,2003-10-07,2003-04,1610612746,1384.0,Los Angeles Clippers,October


In [224]:
monthly = pd.read_csv("monthly_team_perf.csv")
monthly = monthly.rename(columns={"Month": "MONTH"})
monthly.drop(["WIN%", "FG%", "3P%", "FT%", "+/-"], axis=1, inplace=True)
monthly.head()

,MONTH,GP,MIN,PTS,W,L,FGM,FGA,3PM,3PA,...,OREB,DREB,REB,AST,TOV,STL,BLK,PF,TEAM,SEASON
0,October,2,101,177,0,2,69,173,10,26,...,23,83,106,40,42,13,18,49,Atlanta Hawks,2003-04
1,November,16,778,1454,6,10,544,1251,71,231,...,185,495,680,316,277,118,97,339,Atlanta Hawks,2003-04
2,December,15,720,1343,3,12,530,1195,49,162,...,181,476,657,301,241,96,74,336,Atlanta Hawks,2003-04
3,January,15,720,1267,6,9,484,1111,51,156,...,160,442,602,291,262,113,73,348,Atlanta Hawks,2003-04
4,February,11,543,1044,4,7,392,920,71,204,...,145,344,489,219,185,100,70,239,Atlanta Hawks,2003-04


In [253]:
updated = pd.DataFrame()
for team in tqdm(monthly.TEAM.unique()):
    for season in monthly.SEASON.unique():
        print(team, season, i)
        temp = monthly[(monthly.TEAM == team) & (monthly.SEASON == season)]
        for i, row in temp.iterrows():
            print(temp.shape[0], i + 1)
            rolled = temp.rolling(i + 1).sum().dropna().iloc[0].to_frame().T
            rolled["MONTH"] = temp.MONTH
            rolled["SEASON"] = temp.SEASON
            rolled["TEAM"] = temp.TEAM
            updated = pd.concat([updated, rolled])

  0%|          | 0/30 [00:00<?, ?it/s]

Atlanta Hawks 2003-04 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
Atlanta Hawks 2004-05 0
6 8


IndexError: single positional indexer is out-of-bounds

In [244]:
updated

,GP,MIN,PTS,W,L,FGM,FGA,3PM,3PA,FTM,...,DREB,REB,AST,TOV,STL,BLK,PF,MONTH,SEASON,TEAM
0,2.0,101.0,177.0,0.0,2.0,69.0,173.0,10.0,26.0,29.0,...,83.0,106.0,40.0,42.0,13.0,18.0,49.0,October,2003-04,Atlanta Hawks
1,18.0,879.0,1631.0,6.0,12.0,613.0,1424.0,81.0,257.0,324.0,...,578.0,786.0,356.0,319.0,131.0,115.0,388.0,November,2003-04,Atlanta Hawks
2,33.0,1599.0,2974.0,9.0,24.0,1143.0,2619.0,130.0,419.0,558.0,...,1054.0,1443.0,657.0,560.0,227.0,189.0,724.0,December,2003-04,Atlanta Hawks
3,48.0,2319.0,4241.0,15.0,33.0,1627.0,3730.0,181.0,575.0,806.0,...,1496.0,2045.0,948.0,822.0,340.0,262.0,1072.0,January,2003-04,Atlanta Hawks
4,59.0,2862.0,5285.0,19.0,40.0,2019.0,4650.0,252.0,779.0,995.0,...,1840.0,2534.0,1167.0,1007.0,440.0,332.0,1311.0,February,2003-04,Atlanta Hawks
5,75.0,3640.0,6840.0,24.0,51.0,2554.0,5956.0,353.0,1086.0,1379.0,...,2318.0,3230.0,1472.0,1249.0,559.0,381.0,1668.0,March,2003-04,Atlanta Hawks


,GP,MIN,PTS,W,L,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,PF
3,48.0,2319.0,4241.0,15.0,33.0,1627.0,3730.0,181.0,575.0,806.0,1047.0,549.0,1496.0,2045.0,948.0,822.0,340.0,262.0,1072.0


In [85]:
monthly[(monthly.TEAM == "Atlanta Hawks")]

,MONTH,GP,MIN,PTS,W,L,WIN%,FGM,FGA,FG%,...,DREB,REB,AST,TOV,STL,BLK,PF,+/-,TEAM,SEASON
0,10,2,101,177,0,2,0.000,69,173,39.9,...,83,106,40,42,13,18,49,-11,Atlanta Hawks,2003
1,11,16,778,1454,6,10,0.375,544,1251,43.5,...,495,680,316,277,118,97,339,-78,Atlanta Hawks,2003
2,12,15,720,1343,3,12,0.200,530,1195,44.4,...,476,657,301,241,96,74,336,-112,Atlanta Hawks,2003
3,1,15,720,1267,6,9,0.400,484,1111,43.6,...,442,602,291,262,113,73,348,-74,Atlanta Hawks,2003
4,2,11,543,1044,4,7,0.364,392,920,42.6,...,344,489,219,185,100,70,239,-54,Atlanta Hawks,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,10,6,288,637,3,3,0.500,253,562,45.0,...,225,305,146,81,38,23,105,2,Atlanta Hawks,2021
123,11,15,720,1681,8,7,0.533,619,1318,47.0,...,527,670,353,173,108,66,261,53,Atlanta Hawks,2021
124,12,14,672,1554,5,9,0.357,554,1224,45.3,...,469,615,327,171,88,68,256,-52,Atlanta Hawks,2021
125,1,15,720,1703,8,7,0.533,610,1261,48.4,...,514,630,385,191,103,64,292,11,Atlanta Hawks,2021


In [ ]:
df.merge(monthly, left_on=)